In [44]:
# Load packages
import pandas as pd
import numpy as np
import lxml.html as lx
import requests
import requests_cache
from newspaper import Article

requests_cache.install_cache("mycache")

In [3]:
# Read the api key. I got this function from Nick's lecture notes from 1/31.
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")
    
key = read_key("../keys/googlenews.txt")

In [41]:
#link = "https://www.nytimes.com/2019/02/28/business/energy-environment/pge-camp-fire.html"
#link = 'https://krcrtv.com/news/butte-county/highway-70-to-be-closed-for-emergency-camp-fire-repairs'
link = 'https://timesofindia.indiatimes.com/world/china/chinese-expert-warns-of-troops-entering-kashmir/articleshow/59516912.cms'
article = Article(link)
article.download()
article.parse()
article.text

'sending its troops to the disputed Doklam area +\n\nputs Indian territory at risk +\n\nBEIJING: A Chinese expert has argued that his country\'s troops would be entitled to enter the Indian side of Kashmir by extending the logic that has permitted Indian troops to enter an area which is disputed by China and Bhutan This is one of the several arguments made by the scholar in an attempt to blame India for. India has responded to efforts by China to build a road in the Doklam area, which falls next to the trijunction connecting Sikkim with Tibet and Bhutan and"Even if India were requested to defend Bhutan\'s territory, this could only be limited to its established territory, not the disputed area," Long Xingchun, director of the Center for Indian Studies at China West Normal University said in an article. "Otherwise, under India\'s logic, if the Pakistani government requests, a third country\'s army can enter the area disputed by India and Pakistan, including India-controlled Kashmir".Chi

In [58]:
# Or use "https://newsapi.org/v2/top-headlines"

response = requests.get("https://newsapi.org/v2/everything", params = {
    "q": "camp fire",
    "sortBy": "relevancy",
    "apiKey": key
})

response.raise_for_status()

r = response.json()

{'status': 'ok',
 'totalResults': 2306,
 'articles': [{'source': {'id': 'the-new-york-times',
    'name': 'The New York Times'},
   'author': 'JILL COWAN',
   'title': 'California Today: California Today: Why ‘the Worst of Both Worlds’ Is in the Forecast',
   'description': 'Friday: Californians will most likely face more extreme weather; Pacific Gas & Electric’s role in the Camp Fire; San Diego treats',
   'url': 'https://www.nytimes.com/2019/03/01/us/california-today-extreme-weather-forecast.html',
   'urlToImage': 'https://static01.nyt.com/images/2019/03/01/us/01campfirecatoday/merlin_151063710_b332153e-e314-410c-8eef-5c41339f26bd-facebookJumbo.jpg',
   'publishedAt': '2019-03-01T14:09:58Z',
   'content': 'Im heading to San Diego this weekend, which feels like a real stroke of luck, in light of this weeks recommendation from Tejal Rao, The Timess California restaurant critic. But even if youre far away, take this as an invitation to treat yourself to your favor… [+1197 chars]'},
  {

In [59]:
# Get google news links for each search term
search_term_links = [
    "https://news.google.com/search?q=camp%20fire&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20death%20toll&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20california&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20paradise&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20map&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20news&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20cleanup&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20debris%20removal&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20pg&e&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20investigation&hl=en-US&gl=US&ceid=US%3Aen"
]

In [76]:
# Get article links
def get_links(url):
    """Gets the article links for one Chronicle article list. The parameter "url" is the url to the 
    article list."""
    
    response = requests.get(url)
    response.raise_for_status()
    html = lx.fromstring(response.text)
    html.make_links_absolute(url)
    
    # Get the links that contain the word "article".
    #links = html.xpath("//div/a/@href")
    links = html.xpath("//*[contains(@class, 'NiLAwe y6IFtc R7GTQ keNKEd j7vNaf nID9nc')]/a/@href")
    #
    
    # Remove duplicate links.
    #links = list(set(links))

    return links

In [82]:
# Get the unique links
l = [get_links(l) for l in search_term_links]
#links_set = set([item for sublist in l for item in sublist])
#list(links_set)
l_all = [item for sublist in l for item in sublist]
len(np.unique(l_all))

703

In [51]:
links = get_links("https://news.google.com/topics/CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEZtYUc1bWVuRnFaaElDWlc0b0FBUAE?hl=en-US&gl=US&ceid=US%3Aen")
links[0]

'https://www.cnn.com/2019/02/08/us/camp-fire-deaths/index.html'

In [55]:
def get_article_text(link):
    article = Article(link)
    article.download()
    article.parse()
    return article.text